In [1]:
import pycqed as pq
import numpy as np
from pycqed.measurement import measurement_control
from pycqed.measurement.sweep_functions import None_Sweep
import pycqed.measurement.detector_functions as det
from qcodes import station
station = station.Station()


Data directory set to: /Users/Adriaan/Documents/Testing/Data
Could not import msvcrt (used for detecting keystrokes)


## Creating an instance of the measurement control

Measurements are controlled through the `MeasurementControl` usually instantiated as `MC`

In [2]:
MC = measurement_control.MeasurementControl('MC',live_plot_enabled=True, verbose=True)
MC.station = station
station.add_component(MC)

'MC'

In [3]:
from pycqed.instrument_drivers.virtual_instruments import instrument_monitor as im 
IM = im.InstrumentMonitor('IM', station)
station.add_component(IM)
# Link the instrument monitor to the MC so that it gets updated in the loop
MC.instrument_monitor('IM')

In [4]:
IM.update()

## Create instruments used in the experiment 
Let's start by creating a dummy instrument called MockParabola. 


In [5]:
from pycqed.instrument_drivers.physical_instruments.dummy_instruments import DummyParHolder
dummy_instrument = DummyParHolder('dummy_instrument')
station.add_component(dummy_instrument)

'dummy_instrument'

## A 1D hard measurement 
A hard measurement is a measurement where the data acquisition loop happens in the **hard**ware.

In [6]:
MC.soft_avg(15)
MC.persist_mode(True)
MC.set_sweep_function(None_Sweep(sweep_control='hard'))
MC.set_sweep_points(np.linspace(0, 10, 30))
MC.set_detector_function(det.Dummy_Detector_Hard(noise=0.5, delay=.02))
dat = MC.run('dummy_hard')
data_set = dat['dset']

Starting measurement: dummy_hard
Sweep function: None_Sweep
Detector function: Dummy_Detector
 100% completed 	elapsed time: 1.0s 	time left: 0.0s


By setting persist_mode = True we can see a copy of the last measurements

In [7]:

MC.set_sweep_function(None_Sweep(sweep_control='hard'))
MC.set_sweep_points(np.linspace(0, 10, 30))
MC.set_detector_function(det.Dummy_Detector_Hard(noise=0.5, delay=.02))
dat2 = MC.run('dummy_hard persistent')
data_set2 = dat2['dset']

Starting measurement: dummy_hard persistent
Sweep function: None_Sweep
Detector function: Dummy_Detector
 100% completed 	elapsed time: 1.5s 	time left: 0.0s


# A simple 1D soft measurement 
A soft measurement is a a measurement where the data acquisition loop occurs in the **soft**ware

In [8]:
dummy_instrument.x(145/134545)
IM.update()

In [9]:
dummy_instrument.delay(.01)
MC.soft_avg(15)
MC.set_sweep_function(dummy_instrument.x)
MC.set_sweep_points(np.linspace(-1,1,30))
dummy_instrument.noise(1)

MC.set_detector_function(dummy_instrument.parabola)
dat = MC.run('1D test')
data_set = dat['dset']

# the second plot will also show the first line
MC.set_sweep_function(dummy_instrument.x)
MC.set_sweep_points(np.linspace(-1,1,30))

dat2= MC.run('1D test-persist')
data_set2 = dat2['dset']

Starting measurement: 1D test
Sweep function: x
Detector function: parabola
 100% completed 	elapsed time: 6.3s 	time left: 0.0s
Starting measurement: 1D test-persist
Sweep function: x
Detector function: parabola
 100% completed 	elapsed time: 6.5s 	time left: 0.0s


You can play around a bit with the options in the MC: 

In [10]:
MC.persist_mode(True) # Turns on and off persistent plotting
MC.verbose(True)
MC.plotting_interval(.2)
MC.live_plot_enabled(True)

# A simple 2D measurement

In [11]:
dummy_instrument.delay(.0001)
MC.soft_avg(4)

sweep_pts = np.linspace(-2, 2, 30)
sweep_pts_2D = np.linspace(-2, 2, 5)

MC.set_sweep_function(dummy_instrument.x)
MC.set_sweep_function_2D(dummy_instrument.y)
MC.set_sweep_points(sweep_pts)
MC.set_sweep_points_2D(sweep_pts_2D)
MC.set_detector_function(dummy_instrument.parabola)
dat=MC.run('test', mode='2D')
data_set = dat['dset']


Starting measurement: test
Sweep function 0: x
Sweep function 1: Sweep_function
Detector function: parabola
 100% completed 	elapsed time: 3.9s 	time left: 0.0s


# 2D combinatioin of a hard inner and soft outer loop
The hard inner loop returns 30 values 

In [12]:
MC.soft_avg(1)
sweep_pts = np.linspace(0, 10, 30)
sweep_pts_2D = np.linspace(0, 10, 30)
MC.set_sweep_function(None_Sweep(sweep_control='hard'))
MC.set_sweep_function_2D(None_Sweep(sweep_control='soft'))
MC.set_sweep_points(sweep_pts)
MC.set_sweep_points_2D(sweep_pts_2D)
MC.set_detector_function(det.Dummy_Detector_Hard(delay=.05, noise=.1))
dat = MC.run('2D_hard', mode='2D')
data_set = dat['dset']

Starting measurement: 2D_hard
Sweep function 0: None_Sweep
Sweep function 1: None_Sweep
Detector function: Dummy_Detector
 100% completed 	elapsed time: 11.4s 	time left: 0.0s


## A Hard measurement that uses soft averaging

The number of soft_averages determines how many times the experiment will be performed. 
Only the averaged data is plotted and saved. 
The number of soft-averages can be set as a parameter of the Measurement Control. 

Will first implement it for 1D hard sweeps (easier) and then follow for combinations of hard and soft sweeps. 

In [13]:
MC.soft_avg(4)
MC.set_sweep_function(None_Sweep(sweep_control='hard'))
MC.set_sweep_points(np.linspace(0, 10, 30))
MC.set_detector_function(det.Dummy_Detector_Hard(noise=1.5, delay=.02))

dat = MC.run('dummy_hard')
data_set = dat['dset']

Starting measurement: dummy_hard
Sweep function: None_Sweep
Detector function: Dummy_Detector
 100% completed 	elapsed time: 0.9s 	time left: 0.0s


# 2D soft averaging

In [16]:
MC.soft_avg(10)
sweep_pts = np.linspace(0, 10, 30)
sweep_pts_2D = np.linspace(0, 10, 5)
MC.set_sweep_function(None_Sweep(sweep_control='hard'))
MC.set_sweep_function_2D(None_Sweep(sweep_control='soft'))
MC.set_sweep_points(sweep_pts)
MC.set_sweep_points_2D(sweep_pts_2D)
MC.set_detector_function(det.Dummy_Detector_Hard(noise=1.5, delay=.001))

dat = MC.run('dummy_hard_2D', mode='2D')
data_set = dat['dset']

Starting measurement: dummy_hard_2D
Sweep function 0: None_Sweep
Sweep function 1: None_Sweep
Detector function: Dummy_Detector
 100% completed 	elapsed time: 13.6s 	time left: 0.0s



## Starting an adaptive measurement 
This example does a 2D optimization over the mock parabola

In [15]:
from pycqed.measurement.optimization import nelder_mead
MC.soft_avg(1)
dummy_instrument
MC.set_sweep_functions([dummy_instrument.x, dummy_instrument.y])
MC.set_adaptive_function_parameters({'adaptive_function':nelder_mead, 
                                    'x0':[-5,-5], 'initial_step': [2.5, 2.5]})
dummy_instrument.noise(.5)
MC.set_detector_function(dummy_instrument.parabola)
dat = MC.run('1D test', mode='adaptive')
data_set = dat['dset']

Starting measurement: 1D test
Sweep function 0: module
Sweep function 1: module
Detector function: parabola
Optimization completed in 1.357s
